In [ ]:
import pandas as pd
import numpy as np
import os
import pickle

In [240]:
class controller():
    def __init__(self):
        self.building_id_list = list(range(0,1449))
        self.year_upper_limit = 2018
        self.year_lower_limit = 2016
        self.service = service()

    def predict(self, building_id, year):
        try:
            assert isinstance(building_id, int), 'building id should be integer'
            assert building_id in self.building_id_list, 'please provide valid building id'
            assert isinstance(year, int), 'year should be integer'
            assert (year >= self.year_lower_limit) & (year <= self.year_upper_limit),\
                   f'year should within {(self.year_lower_limit, self.year_upper_limit)}'
        except AssertionError as e:
            return e
        output = self.service.predict(building_id, year)
        return output

class dao():
    def __init__(self):
        self.conn = sqlite3.connect('data/input_data/ashrae.db')

    def get_meter_list(self, building_id):
        query = """
                SELECT meter FROM building_meter_map
                WHERE building_id==?;
                """
        cursor = self.conn.execute(query, (building_id,))
        self.conn.commit()
        return (cursor.fetchall())

    def get_preprocessed_data(self, building_id, year, meter):
        feature = ['air_temperature', 
                   'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
                   'air_temperature_max_past_18',
                   'air_temperature_min_past_18', 'air_temperature_mean_past_18',
                   'air_temperature_median_past_18', 'air_temperature_std_past_18',
                   'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
                   'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
                   'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
                   'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
                   'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
                   'sin_hour', 'sin_month', 
                   'meter', 'building_id', 'quarter',
                   'site_id', 'primary_use', 'floor_count',
                   'log_sq_ft', 'month',
                   'dayofweek', 'hour']

        
        feature_str = f"""air_temperature, 
                            dew_temperature, precip_depth_1_hr, sea_level_pressure,
                            air_temperature_max_past_18,
                            air_temperature_min_past_18, air_temperature_mean_past_18,
                            air_temperature_median_past_18, air_temperature_std_past_18,
                            cloud_coverage_max_past_18, cloud_coverage_min_past_18,
                            cloud_coverage_mean_past_18, cloud_coverage_median_past_18,
                            cloud_coverage_std_past_18, dew_temperature_max_past_18,
                            dew_temperature_min_past_18, dew_temperature_mean_past_18,
                            dew_temperature_median_past_18, dew_temperature_std_past_18,
                            sin_hour, sin_month, {meter} AS meter, building_id, quarter,
                            building_metadata.site_id, primary_use, floor_count,
                            log_sq_ft, month, dayofweek, hour
                        """
        query = """
                    SELECT 
                        {}
                    FROM building_metadata
                    INNER JOIN weather_data ON building_metadata.site_id==weather_data.site_id
                    WHERE building_metadata.building_id==? AND strftime('%Y', weather_data.timestamp)==?;
                """.format(feature_str)
        cursor = self.conn.execute(query, (building_id,str(year)))
        df = pd.DataFrame(cursor.fetchall(), columns=feature)
        self.conn.commit()
        return df

class service():
    def __init__(self):
        self.model_0_1 = pickle.load(open('model/model_0_1.pkl', 'rb'))
        self.model_0_2 = pickle.load(open('model/model_0_2.pkl', 'rb'))
        self.model_1 = pickle.load(open('model/model_1.pkl', 'rb'))
        self.model_2 = pickle.load(open('model/model_2.pkl', 'rb'))
        self.model_3 = pickle.load(open('model/model_3.pkl', 'rb'))
        self.building_sample_1 = pickle.load(open('model/building_sample_1.pkl', 'rb'))
        self.building_sample_2 = pickle.load(open('model/building_sample_2.pkl', 'rb'))
        self.feature = pickle.load(open('model/feature.pkl', 'rb'))
        self.dao = dao()
    def predict(self, building_id, year):
        #load data
        meter_list = self.dao.get_meter_list(building_id)
        output = dict()
        for meter_type in meter_list:
            meter = meter_type[0]
            #get_processed_data
            preprocessed_data = self.dao.get_preprocessed_data(building_id, year, meter)
            if meter == 0:
                if building_id in self.building_sample_1:
                    output[meter] = self.model_0_1.predict(preprocessed_data)
                else:
                    output[meter] = self.model_0_2.predict(preprocessed_data)
            elif meter == 1:
                output[meter] = self.model_1.predict(preprocessed_data)
            elif meter == 2:
                output[meter] = self.model_2.predict(preprocessed_data)
            elif meter == 3:
                output[meter] = self.model_3.predict(preprocessed_data)
        return output

In [241]:
input_obj = controller()

In [245]:
feature1 = ['air_temperature', 'dew_temperature', 'precip_depth_1_hr',
                   'sea_level_pressure', 'air_temperature_max_past_18',
                   'air_temperature_min_past_18', 'air_temperature_mean_past_18',
                   'air_temperature_median_past_18', 'air_temperature_std_past_18',
                   'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
                   'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
                   'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
                   'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
                   'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
                   'sin_hour', 'sin_month', 'meter', 'building_id', 'quarter',
                   'site_id', 'primary_use', 'floor_count',
                   'log_sq_ft', 'dayofweek', 'hour']
len(feature1)                   

30

In [246]:
feature2 = ['air_temperature', 
           'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
           'air_temperature_max_past_18',
           'air_temperature_min_past_18', 'air_temperature_mean_past_18',
           'air_temperature_median_past_18', 'air_temperature_std_past_18',
           'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
           'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
           'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
           'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
           'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
           'sin_hour', 'sin_month', 
           'meter', 'building_id', 'quarter',
           'site_id', 'primary_use', 'floor_count',
           'log_sq_ft', 'month',
           'dayofweek', 'hour']

len(feature2)

31

In [247]:
set(feature2) - set(feature1)

{'month'}

In [242]:
result = input_obj.predict(0, 2017)

ValueError: ignored

In [234]:
result

{}

In [1]:
import sqlite3

In [2]:
import os

In [7]:
import pickle

In [3]:
os.chdir('/content/drive/MyDrive/AppliedAI/self_case_study_1/ashrae_energy_prediction')

In [5]:
os.listdir('data/fe_data')

['fe_weather_train_df.pkl',
 'fe_weather_test_df.pkl',
 'fe_building_metadata_df.pkl']

In [8]:
fe_building_data = pickle.load(open('data/fe_data/fe_building_metadata_df.pkl', 'rb'))
fe_building_data.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count,log_sq_ft,year_built_label,floor_sq_feet
0,0,0,1,7432,109,1,8.914062,5,8.914062
1,0,1,1,2720,105,1,7.910156,5,7.910156
2,0,2,1,5376,92,1,8.585938,4,8.585938
3,0,3,1,23685,103,1,10.070312,5,10.070312
4,0,4,1,116607,76,1,11.664062,4,11.664062


In [105]:
fe_weather_train_df = pickle.load(open('data/fe_data/fe_weather_train_df.pkl', 'rb'))
fe_weather_train_df.head(10)

,timestamp,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,air_temperature_max_past_18,...,precip_depth_1_hr_mean_past_18,precip_depth_1_hr_median_past_18,precip_depth_1_hr_std_past_18,month,dayofweek,hour,quarter,sin_hour,sin_month,sin_wind_direction
0,2016-01-01 00:00:00,0,19.406250,6.0,19.40625,0.0,1019.5,0,0.0,19.406250,...,0.000000,0.0,0.707031,1,4,0,1,-0.258789,0.382568,1.000000
1,2016-01-01 01:00:00,0,21.093750,6.0,21.09375,-1.0,1019.5,0,0.0,21.093750,...,-0.500000,-0.5,0.707031,1,4,1,1,-0.500000,0.382568,1.000000
2,2016-01-01 02:00:00,0,21.093750,6.0,21.09375,0.0,1019.0,4,1.5,21.093750,...,-0.333252,0.0,0.577148,1,4,2,1,-0.707031,0.382568,-0.866211
3,2016-01-01 03:00:00,0,20.593750,6.0,20.00000,0.0,1018.0,0,0.0,21.093750,...,-0.250000,0.0,0.500000,1,4,3,1,-0.866211,0.382568,1.000000
4,2016-01-01 04:00:00,0,21.093750,6.0,20.59375,0.0,1019.0,6,1.5,21.093750,...,-0.199951,0.0,0.447266,1,4,4,1,-0.965820,0.382568,0.342041
5,2016-01-01 05:00:00,0,21.093750,6.0,20.59375,-1.0,1018.5,0,0.0,21.093750,...,-0.333252,0.0,0.516602,1,4,5,1,-1.000000,0.382568,1.000000
6,2016-01-01 06:00:00,0,20.593750,6.0,20.00000,0.0,1019.0,0,0.0,21.093750,...,-0.285645,0.0,0.488037,1,4,6,1,-0.965820,0.382568,1.000000
7,2016-01-01 07:00:00,0,18.906250,6.0,18.90625,0.0,1019.0,4,1.5,21.093750,...,-0.250000,0.0,0.462891,1,4,7,1,-0.866211,0.382568,-0.866211
8,2016-01-01 08:00:00,0,20.000000,6.0,20.00000,0.0,1019.5,0,0.0,21.093750,...,-0.222168,0.0,0.440918,1,4,8,1,-0.707031,0.382568,1.000000
9,2016-01-01 09:00:00,0,21.703125,6.0,21.09375,0.0,1020.0,0,0.0,21.703125,...,-0.199951,0.0,0.421631,1,4,9,1,-0.500000,0.382568,1.000000


In [12]:
fe_weather_test_df = pickle.load(open('data/fe_data/fe_weather_test_df.pkl', 'rb'))
fe_weather_test_df.head()

,timestamp,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,air_temperature_max_past_18,...,precip_depth_1_hr_mean_past_18,precip_depth_1_hr_median_past_18,precip_depth_1_hr_std_past_18,month,dayofweek,hour,quarter,sin_hour,sin_month,sin_wind_direction
0,2017-01-01 00:00:00,0,15.601562,2.0,12.796875,0.0,1022.0,2,2.099609,15.601562,...,0.0,0.0,0.0,1,6,0,1,-0.258789,0.382568,-0.642578
1,2017-01-01 01:00:00,0,15.000000,0.0,12.796875,0.0,1021.5,3,1.500000,15.601562,...,0.0,0.0,0.0,1,6,1,1,-0.500000,0.382568,-0.866211
2,2017-01-01 02:00:00,0,15.000000,2.0,13.296875,0.0,1021.5,0,0.000000,15.601562,...,0.0,0.0,0.0,1,6,2,1,-0.707031,0.382568,1.000000
3,2017-01-01 03:00:00,0,13.296875,0.0,12.203125,0.0,1021.5,0,0.000000,15.601562,...,0.0,0.0,0.0,1,6,3,1,-0.866211,0.382568,1.000000
4,2017-01-01 04:00:00,0,12.203125,4.0,11.703125,0.0,1021.5,0,0.000000,15.601562,...,0.0,0.0,0.0,1,6,4,1,-0.965820,0.382568,1.000000


In [14]:
import pandas as pd

In [ ]:
weather_data = pd.concat([fe_weather_train_df,fe_weather_test_df], ignore_index=True)
weather_data.head()

In [59]:
conn = sqlite3.connect('data/input_data/ashrae.db')

In [10]:
fe_building_data.to_sql(name='building_metadata', con=conn, if_exists='fail',
                        index=False)

In [83]:
weather_data.to_sql(name='weather_data', con=conn, if_exists='replace',
                    index=False)

In [145]:
df = pickle.load(open('data/input_data/train_df.pkl', 'rb'))
df.head(2)

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0


In [149]:
building_meter_map = df[['building_id', 'meter']].drop_duplicates(ignore_index=True)

In [150]:
building_meter_map.to_sql(name='building_meter_map', con=conn, if_exists='fail',
                          index=False)

In [61]:
c = conn.cursor()

In [136]:
str([1,2,3])[1:-1]

'1, 2, 3'

In [132]:

c.fetchall()

[(0, 0, 19.40625, '2016-01-01 00:00:00'),
 (0, 0, 21.09375, '2016-01-01 01:00:00'),
 (0, 0, 21.09375, '2016-01-01 02:00:00'),
 (0, 0, 20.59375, '2016-01-01 03:00:00'),
 (0, 0, 21.09375, '2016-01-01 04:00:00')]

In [155]:
query = """
        SELECT meter FROM building_meter_map
        WHERE building_id==?;
        """
cursor = conn.execute(query, (0,))

In [191]:
meter = 0
feature = ['air_temperature', 'dew_temperature', 'precip_depth_1_hr',
                   'sea_level_pressure', 'air_temperature_max_past_18',
                   'air_temperature_min_past_18', 'air_temperature_mean_past_18',
                   'air_temperature_median_past_18', 'air_temperature_std_past_18',
                   'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
                   'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
                   'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
                   'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
                   'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
                   'sin_hour', 'sin_month', 'meter', 'building_id', 'quarter',
                   'site_id', 'primary_use', 'floor_count',
                   'log_sq_ft', 'dayofweek', 'hour']
feature_str = f"""air_temperature, 
                    dew_temperature, precip_depth_1_hr, sea_level_pressure,
                    air_temperature_max_past_18,
                    air_temperature_min_past_18, air_temperature_mean_past_18,
                    air_temperature_median_past_18, air_temperature_std_past_18,
                    cloud_coverage_max_past_18, cloud_coverage_min_past_18,
                    cloud_coverage_mean_past_18, cloud_coverage_median_past_18,
                    cloud_coverage_std_past_18, dew_temperature_max_past_18,
                    dew_temperature_min_past_18, dew_temperature_mean_past_18,
                    dew_temperature_median_past_18, dew_temperature_std_past_18,
                    sin_hour, sin_month, {meter} AS meter, building_id, quarter,
                    building_metadata.site_id, primary_use, floor_count,
                    log_sq_ft, dayofweek, hour
                """
query = """
            SELECT 
                {}
            FROM building_metadata
            INNER JOIN weather_data ON building_metadata.site_id==weather_data.site_id
            WHERE building_metadata.building_id==? AND strftime('%Y', weather_data.timestamp)==?;
        """.format(feature_str)
cursor = conn.execute(query, (0,str(2016)))
df = pd.DataFrame(cursor.fetchall(), columns=feature)
df.head()

,air_temperature,dew_temperature,precip_depth_1_hr,sea_level_pressure,air_temperature_max_past_18,air_temperature_min_past_18,air_temperature_mean_past_18,air_temperature_median_past_18,air_temperature_std_past_18,cloud_coverage_max_past_18,...,sin_month,meter,building_id,quarter,site_id,primary_use,floor_count,log_sq_ft,dayofweek,hour
0,19.40625,19.40625,0.0,1019.5,19.40625,19.40625,19.406250,19.40625,1.202148,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,0
1,21.09375,21.09375,-1.0,1019.5,21.09375,19.40625,20.250000,20.25000,1.202148,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,1
2,21.09375,21.09375,0.0,1019.0,21.09375,19.40625,20.531250,21.09375,0.981445,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,2
3,20.59375,20.00000,0.0,1018.0,21.09375,19.40625,20.546875,20.84375,0.802246,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,3
4,21.09375,20.59375,0.0,1019.0,21.09375,19.40625,20.656250,21.09375,0.736816,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,4


In [199]:
meter = 0
feature = ['air_temperature', 'dew_temperature', 'precip_depth_1_hr',
                   'sea_level_pressure', 'air_temperature_max_past_18',
                   'air_temperature_min_past_18', 'air_temperature_mean_past_18',
                   'air_temperature_median_past_18', 'air_temperature_std_past_18',
                   'cloud_coverage_max_past_18', 'cloud_coverage_min_past_18',
                   'cloud_coverage_mean_past_18', 'cloud_coverage_median_past_18',
                   'cloud_coverage_std_past_18', 'dew_temperature_max_past_18',
                   'dew_temperature_min_past_18', 'dew_temperature_mean_past_18',
                   'dew_temperature_median_past_18', 'dew_temperature_std_past_18',
                   'sin_hour', 'sin_month', 'meter', 'building_id', 'quarter',
                   'site_id', 'primary_use', 'floor_count',
                   'log_sq_ft', 'dayofweek', 'hour']
feature_str = f"""air_temperature, 
                    dew_temperature, precip_depth_1_hr, sea_level_pressure,
                    air_temperature_max_past_18,
                    air_temperature_min_past_18, air_temperature_mean_past_18,
                    air_temperature_median_past_18, air_temperature_std_past_18,
                    cloud_coverage_max_past_18, cloud_coverage_min_past_18,
                    cloud_coverage_mean_past_18, cloud_coverage_median_past_18,
                    cloud_coverage_std_past_18, dew_temperature_max_past_18,
                    dew_temperature_min_past_18, dew_temperature_mean_past_18,
                    dew_temperature_median_past_18, dew_temperature_std_past_18,
                    sin_hour, sin_month, {meter} AS meter, building_id, quarter,
                    building_metadata.site_id, primary_use, floor_count,
                    log_sq_ft, dayofweek, hour
                """
query = """
            SELECT 
                {}
            FROM building_metadata
            INNER JOIN weather_data ON building_metadata.site_id==weather_data.site_id
            WHERE building_metadata.building_id==? AND strftime('%Y', weather_data.timestamp)==?;
        """.format(feature_str)
cursor = conn.execute(query, (0,str(2016)))
df = pd.DataFrame(cursor.fetchall(), columns=feature)
df.head()

,air_temperature,dew_temperature,precip_depth_1_hr,sea_level_pressure,air_temperature_max_past_18,air_temperature_min_past_18,air_temperature_mean_past_18,air_temperature_median_past_18,air_temperature_std_past_18,cloud_coverage_max_past_18,...,sin_month,meter,building_id,quarter,site_id,primary_use,floor_count,log_sq_ft,dayofweek,hour
0,19.40625,19.40625,0.0,1019.5,19.40625,19.40625,19.406250,19.40625,1.202148,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,0
1,21.09375,21.09375,-1.0,1019.5,21.09375,19.40625,20.250000,20.25000,1.202148,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,1
2,21.09375,21.09375,0.0,1019.0,21.09375,19.40625,20.531250,21.09375,0.981445,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,2
3,20.59375,20.00000,0.0,1018.0,21.09375,19.40625,20.546875,20.84375,0.802246,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,3
4,21.09375,20.59375,0.0,1019.0,21.09375,19.40625,20.656250,21.09375,0.736816,6.0,...,0.382568,0,0,1,0,1,1,8.914062,4,4


In [192]:
df.shape

(8784, 30)

In [156]:
df = pd.DataFrame(cursor.fetchall(), columns=feature)


[(0,)]

In [79]:
conn.commit()

In [200]:
conn.close()